# Oasis Montaj script generator
 _Because I hate clicking_

1. Create an adequately sized geodatbase in Oasis Montaj
1. Import ASEG-GDF and deal with coordinate systems until you're in your projection of choice (e.g. GDA94_MGA55)
1. Run below cells, specifying your own survey_gdb_path and parameters of choice

In [ ]:
import logging
import re
from pathlib import Path
import numpy as np

logging.basicConfig()

survey_gdb_path = Path(r"c:\Luke\PhD\oasis montaj\ArbSR\p738\p738.gdb")
x_min = 420000 # Override calculated "xmin" - note underscore in var names.
nominal_line_spacing = 250  # m
decimation_factors = [1, 2, 3, 4]  # for n in d.factors: Keep every nth line
# xmin, ymin, xmax, ymax = (356800,6124400,504350,6350900,)  # W, S, E, N / xmin, ymin, xmax, ymax



In [ ]:
def duplicate_db_script(
    survey_gdb_path, decimation_factors=[1], lineType_ch="lineType"
):
    """Create a Oasis Montaj script which duplicates a database n times
    and exports a lineType stats report."""

    gdb = Path(survey_gdb_path)
    ext = ".gdb"
    output_gdb_gs = gdb.parent / f"{gdb.stem}_stats_duplicate.gs"

    text = (  # Header
        "/-------------------------------------------------------------------------\n"
        "/ LSgeo 2022\n"
        "/-------------------------------------------------------------------------\n"
    )

    text += (  # Stats File export
        f'CURRENT        Database,"{str(survey_gdb_path)}"\n'
        f'SETINI         XYZSTAT.CHAN="{lineType_ch}"\n'
        f'SETINI         XYZSTAT.FILE=".\\\\{gdb.stem}_stats.txt"\n'
        f'SETINI         XYZSTAT.SHOWLOG="1"\n'
        f'SETINI         EDIT.FILE=".\\\\{gdb.stem}_stats.txt"\n'
        f'SETINI         EDIT.CHILD="Yes"\n'
        f'SETINI         EDIT.CHILD=""\n'
        f'SETINI         EDIT.WAIT=""\n'
        f"GX             xyzstat.gx\n"
        f'CURRENT        Database,"{gdb.with_suffix(ext)}"\n'
    )

    for i in decimation_factors:  # gdb section duplicates
        num = f"_{i}"

        text += (
            f'SETINI         SAVEGDB.UNLOAD="1"\n'
            f'SETINI         SAVEGDB.NAME=".\\\\{gdb.stem}{num + ext}"\n'
            f'SETINI         SAVEGDB.COMPRESSION="2"\n'
            f'SETINI         DBSUBSET.NAME="{str(gdb.with_name(gdb.stem))}{num + ext}"\n'
            f'SETINI         DBSUBSET.LINES="All"\n'
            f'SETINI         DBSUBSET.CHANNELS="All"\n'
            f'SETINI         DBSUBSET.MASK=""\n'
            f'SETINI         DBSUBSET.DIALOG="NO"\n'
            f'SETINI         DBSUBSET.COMPRESSION="2"\n'
            f'SETINI         DBSUBSET.SAVE_AS="1"\n'
            f'SETINI         DBSUBSET.DIALOG="Yes"\n'
            f'SETINI         DBSUBSET.SAVE_AS="0"\n'
            f"GX             savegdb.gx\n"
        )

    text += (  # Footer
        "/-------------------------------------------------------------------------\n"
        "/ LSgeo\n"
        "/-------------------------------------------------------------------------\n"
    )
    output_gdb_gs.write_text(text)
    print(output_gdb_gs)


duplicate_db_script(survey_gdb_path, decimation_factors=decimation_factors)


4. In Oasis Montaj, run the gs script generated by the above cell to generate a stats file and duplicate the database for the decimation factors of interest
5. Execute the below cells and continue from 6. below.

In [ ]:
 #TODO preserve edge lines and select based on edge as line "0"
def create_sel_file(
    survey_gdb_path,
    out_prefix="decimated",
    tie_line_type=4,
    df=1,  # decimation  factor
    drop_tielines=False,
    use_naive_line_selection=False,
    use_line_number_selection=True,
    drop_specific_lines=[]
):
    """Prepare a selection file for Oasis Montaj, based on all available lines

    By default, tie lines "type 4" are kept,  so at Decimation Factor == 1,
    the function will do nothing (i.e. select No lines). At d.f. n > 1, every
    nth line will be "selected" (i.e. written to file)

    The option to naively select every nth line is presented, but this may fail
    if there are complex flight lines. In this case (and by default), lines are
    selected by attempting to interpret the naming scheme.

    LSgeo 2021
    """

    gdb = Path(survey_gdb_path)
    stats_file = gdb.with_name(f"{gdb.stem}_stats.txt")
    if not stats_file.exists():
        raise FileNotFoundError("Export lineType channel report from OM first!")

    ch_sec_summ = stats_file.read_text().split("ALL")[1].split("\n")[0]
    exponent_num = re.compile(r"([0-9.+e\-]+)\s+")
    xmin, xmax, ymin, ymax, _ = np.array(
        re.findall(exponent_num, ch_sec_summ), dtype=float
    )

    for i in [[xmin, "xmin"], [xmax, "xmax"], [ymin, "ymin"], [ymax, "ymax"]]:
        if abs(i[0]) > 1e10:
            logging.error(
                f"Value of {i[0]} is out of range for {i[1]}, determine manually"
            )

    # Get the min, max, average section of statistics report
    ch_sec = stats_file.read_text().split(
        "------------   -------        -------        ----           --------       ----------- ------------"
    )[-1]

    all_lines = []
    for line in ch_sec.split("\n"):  # Split by line
        if ":" in line:  # Only care about line number rows, not blank or header
            for i in line.split(","):
                line_name = i.split("      ")[0]
                line_type = i.split("              ")[2]
                all_lines.append([f'"{line_name[:-2]}"', int(line_type)])
                if "." in line_type:  # decimal point from average
                    raise NotImplementedError(
                        "Line probably contains mixed lineTypes, mad sorry bro"
                    )
    all_lines = sorted(all_lines)

    if use_line_number_selection:
        # Line numbers are n * I + m * J digits, e.g. IIIJ, 3 * "I" + 1 * "J", 
        # where III are same coordinate axis and J is line number on that axis. 
        # Separate Flight and tie lines. Drop extra chars "'" and "L". Split last digit

        flight_lines = np.array(
            [
                [int(l[0][2:-1][:-1]), int(l[0][2:-1][-1])]
                for l in all_lines
                if l[1] == 2
            ]
        )
        tie_lines = np.array(
            [
                [int(l[0][2:-1][:-1]), int(l[0][2:-1][-1])]
                for l in all_lines
                if l[1] == 4
            ]
        )  # Technically don't need to handle this the same way as flight_lines
        # but TODO when I have time to make it simple

        sel_mask = np.concatenate((flight_lines[:, 0] % df, tie_lines[:, 0] % 1))

        # Select line names from all_lines based on modulo math mask above
        sel_lines = list(np.array(all_lines)[:, 0][sel_mask.astype(bool)])
        selection_to_delete = "\n".join(sel_lines)

    else:
        decimation_counter = 0
        selection_to_delete = ""

        for line in all_lines:
            if drop_tielines and line[1] == tie_line_type:
                logging.info(f"Will delete tie Line {line[0]}")
                selection_to_delete += f"{line[0]}\n"
                continue  # Handle tie lines independent of decimation

            if use_naive_line_selection and decimation_counter % df != 0:
                logging.info(f"Will delete Line {decimation_counter}: {line[0]}")
                selection_to_delete += f"{line[0]}\n"
                decimation_counter += 1  # Start at line 0 with this at end of loop

    if drop_specific_lines:
        selection_to_delete += "\n" + "\n".join(drop_specific_lines)

    sel_file = stats_file.with_name(f"{gdb.stem}_{out_prefix}_{df}.sel")
    sel_file.write_text(selection_to_delete)
    print(sel_file.absolute())

    return xmin, xmax, ymin, ymax


def create_sel_and_del_gs(
    survey_gdb_path,
    decimation_factors,
    out_prefix="decimated",
):

    gdb = Path(survey_gdb_path)
    ext = ".gdb"
    output_gs = gdb.parent / f"{gdb.stem}_sel_and_del.gs"

    text = (
        "/-------------------------------------------------------------------------\n"
        "/ LSgeo\n"
        "/-------------------------------------------------------------------------\n"
    )

    for decimation_factor in decimation_factors:
        sf = f"{gdb.stem}_{out_prefix}_{decimation_factor}.sel"
        num = f"_{decimation_factor}"
        text += (
            f'CURRENT        Database,"{str(gdb.with_name(gdb.stem))}{num + ext}"\n'
            f'SETINI         SELGET.FILE=".\\\\{sf}"\n'
            f"GX             selget.gx\n"
            f'CURRENT        Database,"{str(gdb.with_name(gdb.stem))}{num + ext}"\n'
            f'SETINI         DELLINES.LINE="L100"\n'
            f'SETINI         DELLINES.OPTION="1"\n'
            f"GX             dellines.gx\n"
            f'CURRENT        Database,"{str(gdb.with_name(gdb.stem))}{num + ext}"\n'
            f"GX             selall.gx\n"
        )

    text += (
        "/-------------------------------------------------------------------------\n"
        "/ LSgeo\n"
        "/-------------------------------------------------------------------------\n"
    )

    output_gs.write_text(text)
    print(output_gs.absolute())


def create_grid_control_file(
    survey_gdb_path,
    out_prefix="decimated",
    decimation_factor=1,
    hr_line_spacing=None,
    along_line_spacing=None,
    trim_edges=False,
    cs_fac=5,  # Default line spacing:cell size ratio
    cs=None,
    xmin="0",
    ymin="0",
    xmax="0",
    ymax="0",
    bclip="0",  # n.b. included in rangrid.con, but not OM help.
    chan="1",
    zb="",
    zm="",
    logopt="",
    logmin="",
    idsf="",
    bkd="",
    srd="",
    iwt="",
    edgclp="",
    wtslp="",  # n.b. included in OM help, but not rangrid.con.
    tol="",
    pastol="",
    itrmax="",
    ti="",
    icgr="",
):

    """Create control file for Minimum Curvature Gridding in Oasis Montaj

    Priority is given to specified args, then python defaults, then OM defaults
    See rangrid.con example in this folder for explanation and OM defaults.
    """

    gdb = Path(survey_gdb_path)

    line_spacing = hr_line_spacing * decimation_factor
    cs = cs or line_spacing / cs_fac  # between 1/5 and 1/2 is good
    bkd = bkd if bkd != "" else 10 * cs
    wtslp = wtslp if wtslp != "" else line_spacing / cs
    # If the data has been collected along traverse lines, it is recommended to set the slope (wtslp) as the number of grid points between the traverse lines

    if trim_edges: # Adjusting extent instead of bclip means both grids have same extent
        xmin += (50 * (hr_line_spacing / cs_fac)) # guess-timated trim factor 50 * HR_cs
        ymin += (50 * (hr_line_spacing / cs_fac))
        xmax -= (50 * (hr_line_spacing / cs_fac))
        ymax -= (50 * (hr_line_spacing / cs_fac))

    if cs % 1 != 0:
        logging.info(
            f"Decimation Factor {decimation_factor} will result in non-decimal cell size. This is supported, but may be undesired."
        )

    con_str = (
        f"{cs}/\n"
        f"{xmin},{ymin},{xmax},{ymax},{bclip}/\n"
        f"{chan},{zb},{zm},{logopt},{logmin}/\n"
        f"{idsf},{bkd},{srd},{iwt},{edgclp},{wtslp}/\n"
        f"{tol},{pastol},{itrmax},{ti},{icgr}/\n"
    )
    out_file = gdb.with_name(
        f"{gdb.stem}_{out_prefix}_{decimation_factor}_{chan}_rangrid.con"
    )
    out_file.write_text(con_str)

    print(out_file.absolute())


In [ ]:
for decimation_factor in decimation_factors:
    logging.info(f"\n\nRunning Decimation Factor {decimation_factor}")

    xmin, xmax, ymin, ymax = create_sel_file(
        survey_gdb_path,
        df=decimation_factor,
        drop_tielines=False,
        drop_specific_lines=['"L201120"', '"L207310"'] #n.b. inclusion of "L..."
    )

    xmin = x_min or xmin

    create_grid_control_file(
        survey_gdb_path,
        xmin=xmin,
        ymin=ymin,
        xmax=xmax,
        ymax=ymax,
        trim_edges=True,
        chan="mag_awagsLevelled",
        decimation_factor=decimation_factor,
        hr_line_spacing=nominal_line_spacing,
        cs_fac=5,
        tol=1,  # Data range ~ -500, 500
        ti=0,
        itrmax=200,
    )

print(f"Used extent:\n    {xmin = }\n    {ymin = }\n    {xmax = }\n    {ymax = }")

create_sel_and_del_gs(
    survey_gdb_path,
    decimation_factors=decimation_factors,
)


6. Review the above selection files, grid control files, and extent.
7. In Oasis Montaj, run the above gs script.  
    a. _Optionally review the decimated line paths_

In [ ]:
def create_gridding_gs(
    survey_gdb_path,
    decimation_factors,
    grid_ch="mag_awagsLevelled",
    out_prefix="decimated",
):

    og_gdb = Path(survey_gdb_path)
    text = "\n"

    for df in decimation_factors:
        # gdb = og_gdb.with_stem(f"{og_gdb.stem}")
        gdb = og_gdb.parent / (f"{og_gdb.stem}_{df}.gdb")
        grid_name = gdb.parent / f"{gdb.stem}.ers(ERM)"
        con_file = (
            gdb.parent / f"{og_gdb.stem}_{out_prefix}_{df}_{grid_ch}_rangrid.con"
        ).name

        text += (
            f'CURRENT        Database,"{gdb}"\n'
            f'SETINI         RGCON.CHAN="{grid_ch}"\n'
            f'SETINI         RGCON.GRDNAME=".\\\\{grid_name.name}"\n'
            f'SETINI         RGCON.CONNAME=".\\\\{con_file}"\n'
            f"GX             rgcon.gx\n"
            f'CURRENT        Grid,"{grid_name}"\n'
        )

    text += "\n"

    grid_gs = og_gdb.with_name(f"{og_gdb.stem}_grid.gs")
    grid_gs.write_text(text)
    print(grid_gs.absolute())


create_gridding_gs(survey_gdb_path, decimation_factors)


Now that you have some lovely ers grids from Oasis Montaj, navigate to utils\tif_to_tensor.ipynb and continue the workflow. You may need to convert these to data tifs (e.g. QGIS export raw data .tif)

In [ ]:
# ## Workflow
# ### Setup - Import and QC your line database.
# 1. Download a line survey from GADDS, etc, in EPSG:4283 (GDA94) _You may use GDA20 as appropriate_
# 1. Import (ASCII) a line dataset into an Oasis Montaj database, using your template (see e.g. ./GA_ASCII_import.i3) _You may need to increase the database size to 12, etc._
#     a. It may be easier to use the ASEG-GDF import wizard.
# 1. Database Tools > Split on Line Channel, _line_
# 1. Coordinates > Set Coordinate System, _GDA94_
# 1. Coordinates > New Projected Coordinate System, e.g. from _GDA94_ to  _GDA94 / MGA Zone 55_
# 1. Coordinates > Set current X,Y,Z... to _long\_mga55_, etc...
# 1. _You may now wish to inspect the _Line Path_ in a _New Map_. We will be modifying these line data._
#     a. Specifically, we will be dropping tie lines, and decimating every nth line.
# 1. _You may now wish to visualise a low resolution grid to check the import. Grid and Image > Grid Data_
# 1. Export a copy of the __lineType__ _Database Tools > Reports > Line/Channel report_, which will be used for the _stats_file_name_ below.


# ### Execution - Execute code, decimate lines, and grid
# 1. Run the below cells, specifying your created stats.txt, survey name, parameters, etc. Check control file looks acceptable
# 1. Create 5 copies of the database, named Pxxx_1, Pxxx_2, Pxxx_3, Pxxx_4, Pxxx_all. The original database serves as a backup copy. _These will be decimated accordingly._
# 1. With the corresponding database selected:
#     1. Delete tie lines using the selection file or the Oasis Montaj _Select/Deselect Type_ feature
#     1. Select the lines to be deleted using _Get Saved Selections_ with the selection file (will incorporate Tie lines if not already deleted). _n.b. this *deselects* lines that will be *kept* for gridding_
#     1. Delete the lines, and select all remaining lines
#     1. _Grid and Image > Gridding > From control file > Minimum Curvature Control File_, select your data channel and file names as appropriate .ers (copy control file path from output below)
